# Segmenting and Clustering Neighborhoods in Toronto

### This is the peer-graded assignment for week three.

In [40]:
# We perform our imports.
import numpy as np
import pandas as pd

In [41]:
# We read in the dataframe from wikipedia.
df_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [42]:
# We locate the correct dataframe.
df = df_list[0]

In [43]:
# We create and apply our filter to get rid of unassigned boroughs.
b_filter = df['Borough'] != 'Not assigned'
df = df[b_filter]
df.reset_index(inplace = True, drop = True)

In [64]:
# We take a quick look at the shape of the dataframe.
df.shape

(103, 3)